In [3]:
import os
from PIL import Image
from torchvision import transforms
import torchvision.transforms.functional as TF
import random
import torch

# 設定參數
input_dir = 'crop/classByhands'
output_dir = 'corp_augmented_data'
augment_per_image = 5  # 每張圖片產生幾張擴增版本
IMG_SIZE = 64  # 統一圖片大小

# 定義擴增方式
def get_augment_transform():
    return transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=20),
        transforms.ColorJitter(brightness=0.4, contrast=0.4),
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.5),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x + 0.05 * torch.randn_like(x)),  # 加一點高斯噪聲
        transforms.Clamp(0, 1),  # 防止超出範圍
        transforms.ToPILImage()
    ])

# 新增 Clamp 函數：避免 Tensor 超出 0~1 範圍
class Clamp(object):
    def __init__(self, min_val, max_val):
        self.min = min_val
        self.max = max_val
    def __call__(self, tensor):
        return torch.clamp(tensor, self.min, self.max)

# 替換 transforms 中的 Clamp 實作
transforms.Clamp = Clamp

# 開始擴增
for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    output_class_path = os.path.join(output_dir, class_name)
    os.makedirs(output_class_path, exist_ok=True)

    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        try:
            image = Image.open(img_path).convert("RGB")
        except:
            print(f"無法開啟圖檔：{img_path}")
            continue

        for i in range(augment_per_image):
            transform = get_augment_transform()
            augmented = transform(image)
            new_name = f"{os.path.splitext(img_name)[0]}_aug{i}.jpg"
            augmented.save(os.path.join(output_class_path, new_name))

print("✅ 擴增完成！圖片已儲存在 corp/augmented_data/ 中")


✅ 擴增完成！圖片已儲存在 corp/augmented_data/ 中
